<a href="https://colab.research.google.com/github/ykato27/ai-demo/blob/main/StridedTransformer_Pose3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GitHub<br>
https://github.com/Vegetebird/StridedTransformer-Pose3D<br>
論文<br>
https://arxiv.org/abs/2103.14304<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/StridedTransformer_Pose3D.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## StridedTransformer-Pose3D

- 概要
    - Strided Transformerは、Transformerベースの2D pose sequenceを活用し、3D pose sequenceを生成する姿勢推定手法です。
    - Strided Transformerでは、Vanilla Transformer Encoder(VTE)を使用し、2D poseシーケンスのlong-rangeの依存関係をモデル化します。この時、シーケンスの冗長性を減らすため、VTEに繋がるレイヤーは、シーケンスの長さを徐々に縮小し、ローカルコンテキストからの情報を集約するためstrided CNNに置き換えられています。

## Githubからソースコード取得

In [1]:
%cd /content

!git clone https://github.com/Vegetebird/StridedTransformer-Pose3D.git

/content
fatal: destination path 'StridedTransformer-Pose3D' already exists and is not an empty directory.


## ライブラリのインストール

In [2]:
%cd /content/StridedTransformer-Pose3D

!pip install --upgrade gdown
!pip install yacs
!pip install filterpy
!pip install einops
!pip install yt-dlp moviepy
!pip install matplotlib==3.0.3
!pip install imageio==2.4.1

/content/StridedTransformer-Pose3D
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 59.1 MB/s eta 0:00:00
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached imageio-2.4.1-py3-none

## ライブラリのインポート

In [3]:
%cd /content/StridedTransformer-Pose3D

import os
from yt_dlp import YoutubeDL
from moviepy.video.fx.resize import resize
from moviepy.editor import (
    VideoFileClip, AudioFileClip, ImageSequenceClip, CompositeAudioClip
)
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

/content/StridedTransformer-Pose3D


# 学習済みモデルのセットアップ

In [4]:
%cd /content/StridedTransformer-Pose3D
!mkdir -p ./checkpoint/pretrained

if not os.path.exists('checkpoint/pretrained/refine_4365.pth'):
  !gdown https://drive.google.com/uc?id=1aDLu0SB9JnPYZOOzQsJMV9zEIHg2Uro7 -O checkpoint/pretrained/refine_4365.pth
if not os.path.exists('checkpoint/pretrained/no_refine_4365.pth'):
  !gdown https://drive.google.com/uc?id=1l63AI9BsNovpfTAbfAkySo9X2MOWgYZH -O checkpoint/pretrained/no_refine_4365.pth

if not os.path.exists('demo/lib/checkpoint/yolov3.weights'):
  !gdown https://drive.google.com/uc?id=1gWZl1VrlLZKBf0Pfkj4hKiFxe8sHP-1C -O demo/lib/checkpoint/yolov3.weights
if not os.path.exists('demo/lib/checkpoint/pose_hrnet_w48_384x288.pth'):
  !gdown https://drive.google.com/uc?id=1CpyZiUIUlEjiql4rILwdBT4666S72Oq4 -O demo/lib/checkpoint/pose_hrnet_w48_384x288.pth

/content/StridedTransformer-Pose3D


# テスト動画のセットアップ

In [5]:
%cd /content/StridedTransformer-Pose3D/demo/video

video_url = 'https://www.youtube.com/watch?v=ChXthSO5J00' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  10#@param {type:"integer"}
end_sec =  20#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

/content/StridedTransformer-Pose3D/demo/video


In [6]:
download_resolution = 360
full_video_path = '/content/StridedTransformer-Pose3D/demo/video/full_video.mp4'
file_name = 'input_clip.mp4'
input_clip_path = '/content/StridedTransformer-Pose3D/demo/video/' + file_name

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

[youtube] Extracting URL: https://www.youtube.com/watch?v=ChXthSO5J00
[youtube] ChXthSO5J00: Downloading webpage
[youtube] ChXthSO5J00: Downloading android player API JSON
[info] ChXthSO5J00: Downloading 1 format(s): 18
Deleting existing file /content/StridedTransformer-Pose3D/demo/video/full_video.mp4
[download] Destination: /content/StridedTransformer-Pose3D/demo/video/full_video.mp4
[download] 100% of    3.35MiB in 00:00:00 at 5.61MiB/s   
Moviepy - Building video /content/StridedTransformer-Pose3D/demo/video/input_clip.mp4.
MoviePy - Writing audio in input_clipTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/StridedTransformer-Pose3D/demo/video/input_clip.mp4



Moviepy - Done !
Moviepy - video ready /content/StridedTransformer-Pose3D/demo/video/input_clip.mp4


# 3D Human Pose Estimation

In [7]:
%cd /content/StridedTransformer-Pose3D

!python demo/vis.py --video {file_name}

/content/StridedTransformer-Pose3D

Generating 2D pose...
100% 300/300 [00:39<00:00,  7.58it/s]
Generating 2D pose successful!

Generating 3D pose...
  7% 20/300 [00:10<02:24,  1.94it/s]/usr/local/lib/python3.9/dist-packages/matplotlib/pyplot.py:507: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  warnings.warn(
100% 300/300 [02:35<00:00,  1.92it/s]
Generating 3D pose successful!

Generating demo...
100% 300/300 [02:35<00:00,  1.92it/s]
Generating demo successful!


In [8]:
input_clip_path = '/content/StridedTransformer-Pose3D/demo/output/input_clip/input_clip.mp4'

clip = VideoFileClip(input_clip_path)
clip_resized = resize(clip, height=420)
clip_resized.ipython_display()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
